# MPIプログラム 課題


## 課題について

課題１、２、３がある。それぞれのGoogleFormに指定されいてる形で、colabのリンクを含めて5月30日までに提出せよ。CoursePowerに提出場所（GoogleForm）が設定されている。

### GoogleColab について2
pythonのコードの実行がブラウザ上でできるサービス。実行する際に計算リソースに割り付けられて実行する
- コード部分 左側の括弧にマウスポインタを持って行くと実行できる
- テキスト部分 マークダウン形式で文章がかける

ファイルからドライブにコピーを保存を選択し、その後は自分のドライブにコピーされるのでそれを実行していく

### C言語の実行について
- ！をつければコマンドを実行でき、C言語のコンパイルもできる
- コード部分に %%file ファイル名 と記述することで、ファイルの中身を記述することができる
- ! gcc test.c
  - test.cをコンパイルし、a.outの実行ファイルを作成する
- ! ./a.out
  - a.outを実行


## 課題１

temp1.cをコンパイルし、10プロセスで実行せよ。実行結果を値の意味も含めて説明せよ。

Colab関連補足
- ! mpicc file.c
  - mpiのプログラムをコンパイルし、実行ファイルを生成する
- ! mpiexec --allow-run-as-root --oversubscribe -np 3 ./a.out
  - これは3プロセスでmpiプログラム ./a.out を実行する


In [34]:
%%file temp1.c
#include <stdio.h>
#include <mpi.h>
int main(){
  int i, my_rank, num_procs;
  MPI_Init(NULL, NULL);
  MPI_Comm_rank(MPI_COMM_WORLD, &my_rank);
  MPI_Comm_size(MPI_COMM_WORLD, &num_procs);

  printf("numprocs:%3d my_rank:%3d \n",num_procs, my_rank);

  MPI_Finalize();
  return 0;
}

Overwriting temp1.c


In [35]:
! mpicc temp1.c
! mpiexec --allow-run-as-root --oversubscribe -np 10 ./a.out

numprocs: 10 my_rank:  9 
numprocs: 10 my_rank:  5 
numprocs: 10 my_rank:  2 
numprocs: 10 my_rank:  3 
numprocs: 10 my_rank:  7 
numprocs: 10 my_rank:  8 
numprocs: 10 my_rank:  4 
numprocs: 10 my_rank:  0 
numprocs: 10 my_rank:  6 
numprocs: 10 my_rank:  1 


## 課題２

1プロセスの時のinner_product.c の実行時間を詳細に分けて計測し、プログラムとしては何倍まで高速化できそうか見積もってみよ。そのため、自分で時間計測関数を挿入せよ。

scatterを読んでいるため、プロセス数がベクトルサイズの約数とならなければならないことに注意せよ。

###実行時間計測場所
メイン関数内のtimer1からtimer4まで計測する。下に書かれているように、時間計測では、MPI_Barrier関数とMPI_Wtime関数を使う。

###プログラム内容と実行時間
- あるプロセスでベクトルを乱数で初期化 （逐次処理）
  - timer1-timer2
- それを全プロセスに分散（逐次処理）
  - timer2-timer3
- 並列に内積計算（並列処理）
  - timer3-timer4





In [36]:
%%file inner_product_mpi.c
#include<stdio.h>
#include<stdlib.h>
#include<mpi.h>
void init_data(double *X, int size)
{
  int i;
  srand(10);
  for(i=0;i<size;i++) X[i]= (double)rand();
}

double inner_product2(double *data1,double  *data2, int n)
{
   double val, nrm;
   int j;
   val = 0.0;
   for(j=0;j<n;j++) val += data1[j]*data2[j];
   MPI_Allreduce(&val, &nrm, 1, MPI_DOUBLE, MPI_SUM, MPI_COMM_WORLD);
   return(nrm);
}

int main()
{
      double s,e,s1,e1,s2,e2,s3,e3;

  double *X,*pX,nrm;
  int n,SIZE=10000000, my_rank, numproc;;
  MPI_Init(NULL, NULL);

  MPI_Barrier(MPI_COMM_WORLD);
  s=MPI_Wtime();

  MPI_Comm_rank(MPI_COMM_WORLD, &my_rank);
  MPI_Comm_size(MPI_COMM_WORLD, &numproc);

  MPI_Barrier(MPI_COMM_WORLD);
  e=MPI_Wtime();

  //---timer 1
  MPI_Barrier(MPI_COMM_WORLD);
  s1=MPI_Wtime();

  if(my_rank==0){
    X=(double *)malloc(sizeof(double)*SIZE);
    init_data(X, SIZE);
  }

  MPI_Barrier(MPI_COMM_WORLD);
  e1=MPI_Wtime();

  //---timer 2
  MPI_Barrier(MPI_COMM_WORLD);
  s2=MPI_Wtime();

  n=SIZE/numproc;// 割り切れる必要あり
  pX = (double*)malloc(sizeof(double) * n);
  MPI_Scatter(X, n, MPI_DOUBLE, pX, n, MPI_DOUBLE, 0, MPI_COMM_WORLD);

  MPI_Barrier(MPI_COMM_WORLD);
  e2=MPI_Wtime();

  //---timer 3
  MPI_Barrier(MPI_COMM_WORLD);
  s3=MPI_Wtime();

  nrm=inner_product2(pX, pX, n);
  if(my_rank==0) printf("length^2 is %f\n", nrm);

  MPI_Barrier(MPI_COMM_WORLD);
  e3=MPI_Wtime();

  //---timer 4

     if (my_rank == 0) {
        printf("Timer1: %lf seconds\n", e - s);
        printf("Timer2: %lf seconds\n", e1 - s1);
        printf("Timer3: %lf seconds\n", e2 - s2);
        printf("Timer4: %lf seconds\n", e3 - s3);
    }

  MPI_Finalize();
}

Overwriting inner_product_mpi.c


In [37]:
! mpicc inner_product_mpi.c
! time mpiexec --allow-run-as-root --oversubscribe -np 1 a.out


length^2 is 15373394046108394074406912.000000
Timer1: 0.000001 seconds
Timer2: 0.257840 seconds
Timer3: 0.079239 seconds
Timer4: 0.035093 seconds

real	0m0.711s
user	0m0.298s
sys	0m0.179s


## 課題３

下のコードは、ランク番号の隣接するプロセス間で、整数1個を交換するプログラムである。隣接するプロセスとは、ここでは連番のプロセス（ランク番号がmy_rank-1,my_rank+1の２プロセス）を意味する。

if文の条件式を適切に設定すると、どんなプロセス数で実行しても動作するようになる。
...内を適切に修正せよ。また、２プロセス間で通信する際に時間計測を行い、プロセス間で整数1個交換するのに、どの程度の時間がかかっているか見積もってみよ。

In [81]:
%%file test.c
#include<stdio.h>
#include<stdlib.h>
#include<mpi.h>

int main()
{
  double start, end, totaltime = 0.0;
  const int N = 10000;
  int my_rank, numproc;
  int val, rval1=-1, rval2=-1;
  MPI_Init(NULL, NULL);
  MPI_Comm_rank(MPI_COMM_WORLD, &my_rank);
  MPI_Comm_size(MPI_COMM_WORLD, &numproc);

  MPI_Status s;
  val = my_rank;

  MPI_Barrier(MPI_COMM_WORLD);
  start = MPI_Wtime();

   for(int i = 0; i < N; i++) {
  // my_rank <--> my_rank-1 data交換
  if(my_rank>0){
    MPI_Send(&val, 1,MPI_INT,my_rank-1, 0,MPI_COMM_WORLD);
    MPI_Recv(&rval1,1,MPI_INT,my_rank-1,0,MPI_COMM_WORLD,&s);
  }

  // my_rank <--> my_rank+1 data交換 >
  if(my_rank < numproc - 1){
    MPI_Send(&val, 1,MPI_INT,my_rank+1, 0,MPI_COMM_WORLD);
    MPI_Recv(&rval2,1,MPI_INT,my_rank+1,0,MPI_COMM_WORLD,&s);
  }
 }
  MPI_Barrier(MPI_COMM_WORLD);
  end = MPI_Wtime();
  totaltime = end - start;
  if (my_rank == 0) {
  printf("Time: %.10lf seconds\n", totaltime / N);}
  printf("%d my_rank:%d %d\n", rval1, val, rval2);
  MPI_Finalize();
}

Overwriting test.c


In [82]:
! mpicc test.c
! mpiexec --allow-run-as-root --oversubscribe -np 2 a.out

Time: 0.0000006455 seconds
-1 my_rank:0 1
0 my_rank:1 -1


# 時間計測方法

特定の部分の時間を計測する方法について説明する
- MPI_Wtime（）という関数がある。
- マルチプロセスで時間計測をするときには、MPI_Barrier関数を呼んで全員が揃ってからタイマーをスタートしもう一度Barrier関数を呼んでストップする。

In [66]:
%%file time_measure_mpi.c
#include<stdio.h>
#include<mpi.h>

int main()
{
  double s,e;
  int my_rank, numproc, i,j=0;
  MPI_Init(NULL, NULL);
  MPI_Comm_rank(MPI_COMM_WORLD, &my_rank);
  MPI_Comm_size(MPI_COMM_WORLD, &numproc);

  //timer 1 ----
  MPI_Barrier(MPI_COMM_WORLD);
  s=MPI_Wtime();

  for(i=0;i<10000000;i++) j++;

  //timer 2 ----
  MPI_Barrier(MPI_COMM_WORLD);
  e=MPI_Wtime();

  //timer 1,2の間の時間の表示
  if(my_rank==0) printf("%lf second \n", e-s);

  MPI_Finalize();
  return 0;
}

Overwriting time_measure_mpi.c


In [67]:
! mpicc time_measure_mpi.c
! mpirun --allow-run-as-root --oversubscribe -np 20 ./a.out

0.317134 second 
